In [ ]:
pip install wikipedia langchain openai elevenlabs gradio

In [ ]:
import os
import soundfile as sf
import numpy as np
import gradio
import openai
import constants
from elevenlabs import generate, play, save
from langchain.chat_models import ChatOpenAI
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

# Load API keys from environment variables
os.environ["OPENAI_API_KEY"] = constants.OPENAI_API_KEY
openai.api_key = constants.OPENAI_API_KEY

In [ ]:
def get_summary_audio(theme, minutes):
    num_words = float(minutes) * 150  # Approx 150 words/min spoken

    # Fetch text from Wikipedia
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    text = wikipedia.run(theme)

    # Send a prompt to summarize the text using ChatGPT
    prompt = f"Summarize the following text in {num_words} words. Text: {text}"
    chat_model = ChatOpenAI()
    summary = chat_model.predict(prompt)

    # Generate speech from the summary using elevenlabs text-to-speech (TTS)
    audio = generate(text=summary, voice='Arnold', model="eleven_monolingual_v1", api_key=constants.ELEVEN_API_KEY)

    # Save the audio as a WAV file
    audio_file_path = f"audio/{theme}_in_{num_words}_words.wav"
    save(audio, audio_file_path)

    # Read the WAV file and format audio data as specified
    audio_data, sample_rate = sf.read(audio_file_path)
    audio_data = (audio_data * 32767).astype(np.int16)
    
    return int(sample_rate), audio_data

def main(theme, minutes):
    return get_summary_audio(theme, minutes)

if __name__ == "__main__":
    demo = gradio.Interface(fn=main, inputs=["text", "text"], outputs="audio", title="Text Tales")
    demo.launch(share=True)